In [32]:
import pandas as pd
import numpy as np
import requests
import bs4
import json
import re
pd.set_option('display.max_rows', 500)

In [129]:
def getWikipediaList(url, table_index):
    
    page = requests.get(url)
    soup = bs4.BeautifulSoup(page.content, 'html.parser')
    tables = soup.find_all('table')
    table = tables[table_index]
    table_headers = []
    row_headers = []
    for tx in table.find_all('th'):
        if tx.has_attr('scope'):
            if tx['scope'] == 'col':
                table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
            if tx['scope'] == 'row':
                row_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
        else:
            table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
    result = pd.DataFrame([[td.text for td in row.find_all('td')] for row in table.tbody.find_all('tr')])
    print(row_headers)
    result.columns = table_headers
    result.replace(r'\n', '', regex = True, inplace = True)
    result.replace(r'[^\x00-\x7F]+', '', regex = True, inplace = True)
    result.drop([0], inplace= True)
    if(len(row_headers)>0):
        result = result.shift(periods=1, axis=1)

        result = result[0:len(result)-1]
        result['Name'] = row_headers
    return result

In [133]:
# THIS CELL IS FOR DEBUGGING THE ABOVE FUYNCTION
url = 'https://en.wikipedia.org/wiki/OECD'
page = requests.get(url)
soup = bs4.BeautifulSoup(page.content, 'html.parser')
tables=soup.find_all('table')
table = tables[11]
table_headers = []
row_headers = []
#print(table)
for tx in table.find_all('th'):
    if tx.has_attr('scope'): #and tx['scope'] == 'col':
        if tx['scope'] == 'row':
            row_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
        else:
            table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
    else:
        table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))


print(table_headers)
result = pd.DataFrame([[td.text for td in row.find_all('td')] for row in table.tbody.find_all('tr')])
result.columns = table_headers
result.replace(r'\s', '', regex = True, inplace = True)
result.drop([0], inplace= True)
result = result.shift(periods=1, axis=1, )
result = result[0:len(result)-1]
#print(len(row_headers))

['Country', 'Application', 'Negotiations', 'Invitation', 'Membership', 'Geographic location', 'Notes']


In [149]:
###### UNFCC
# EU
# NATO
# OECD
# G7
# UN
# OSCE
# BIS
# COE
# ILO
# INTERPOL
uncfcc_countries = getWikipediaList(url = 'https://en.wikipedia.org/wiki/List_of_parties_to_the_United_Nations_Framework_Convention_on_Climate_Change', table_index=0)
eu_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_state_of_the_European_Union', table_index=1)
nato_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_states_of_NATO', 0)
oecd_countries = getWikipediaList('https://en.wikipedia.org/wiki/OECD', 11)
g7_countries = getWikipediaList('https://en.wikipedia.org/wiki/Group_of_Seven', 4)
un_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_states_of_the_United_Nations', 1)
# eu standardizationz
eu_countries = eu_countries.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
eu_countries = eu_countries.apply(lambda x: x.str.replace("\[.\]", "") if x.dtype == "object" else x)
#un standardization
un_countries = un_countries.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
un_countries = un_countries.apply(lambda x: x.str.replace("\[.*\]", "") if x.dtype == "object" else x)


[]
[' Austria', ' Belgium', ' Bulgaria', ' Croatia', ' Cyprus[a]', ' Czech Republic', ' Denmark[d]', ' Estonia', ' Finland[e]', ' France[f]', ' Germany', ' Greece', ' Hungary', ' Ireland', ' Italy', ' Latvia', ' Lithuania', ' Luxembourg', ' Malta', ' Netherlands[i]', ' Poland', ' Portugal[k]', ' Romania', ' Slovakia', ' Slovenia', ' Spain[l]', ' Sweden']
[]
[]
[]
[]


In [150]:
un_countries['Member state'].values


array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Plurinational State of Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo', 'Costa Rica', "Cte d'Ivoire",
       'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia'

In [151]:
coordinates = pd.read_csv('assets/world_country_and_usa_states_latitude_and_longitude_values.csv')
coordinates = coordinates[['country_code', 'latitude', 'longitude', 'country']]
coordinates['in_eu'] = np.where(coordinates['country'].isin(eu_countries['Name'].values), True, False)
coordinates['in_uncfcc'] = np.where(coordinates['country'].isin(uncfcc_countries['State'].values), True, False)
coordinates['in_nato'] = np.where(coordinates['country'].isin(nato_countries['Member state'].values), True, False)
coordinates['in_oecd'] = np.where(coordinates['country'].isin(oecd_countries['Country'].values), True, False)
coordinates['in_g7'] = np.where(coordinates['country'].isin(g7_countries['Member'].values), True, False)
coordinates['in_un'] = np.where(coordinates['country'].isin(un_countries['Member state'].values), True, False)
coordinates

,country_code,latitude,longitude,country,in_eu,in_uncfcc,in_nato,in_oecd,in_g7,in_un
0,AD,42.546245,1.601554,Andorra,False,True,False,False,False,True
1,AE,23.424076,53.847818,United Arab Emirates,False,True,False,False,False,True
2,AF,33.939110,67.709953,Afghanistan,False,True,False,False,False,True
3,AG,17.060816,-61.796428,Antigua and Barbuda,False,True,False,False,False,True
4,AI,18.220554,-63.068615,Anguilla,False,False,False,False,False,False
5,AL,41.153332,20.168331,Albania,False,True,True,False,False,True
6,AM,40.069099,45.038189,Armenia,False,True,False,False,False,True
7,AN,12.226079,-69.060087,Netherlands Antilles,False,False,False,False,False,False
8,AO,-11.202692,17.873887,Angola,False,True,False,False,False,True
9,AQ,-75.250973,-0.071389,Antarctica,False,False,False,False,False,False


In [152]:
print("EU Countries: {}".format(len(coordinates[coordinates['in_eu'] == True])))
print("UNCFCC Countries: {}".format(len(coordinates[coordinates['in_uncfcc'] == True])))
print("nato Countries: {}".format(len(coordinates[coordinates['in_nato'] == True])))
print("oecd Countries: {}".format(len(coordinates[coordinates['in_oecd'] == True])))
print("g7 Countries: {}".format(len(coordinates[coordinates['in_g7'] == True])))
print("UN Countries: {}".format(len(coordinates[coordinates['in_un'] == True])))

EU Countries: 27
UNCFCC Countries: 187
nato Countries: 28
oecd Countries: 37
g7 Countries: 7
UN Countries: 169


In [128]:
g7_countries

,Member,Trade mil. USD (2014),Nom. GDP mil. USD (2019),PPP GDP mil. USD (2019),Nom. GDP per capita USD (2019),PPP GDP per capita USD (2019),HDI (2017),Population (2014),Permanent members of UN Security Council,DAC,OECD,Economic classification (IMF)
1,Canada\n,"947,200\n","1,736,426\n","1,920,997\n","46,271\n","51,190\n",0.926\n,"35,467,000\n",N\n,Y\n,Y\n,Advanced\n
2,France\n,"1,212,300\n","2,715,818\n","3,228,039\n","41,896\n","49,798\n",0.901\n,"63,951,000\n",Y\n,Y\n,Y\n,Advanced\n
3,Germany\n,"2,866,600\n","3,861,550\n","4,672,006\n","46,472\n","56,226\n",0.936\n,"80,940,000\n",N\n,Y\n,Y\n,Advanced\n
4,Italy\n,"948,600\n","2,001,466\n","2,665,524\n","33,159\n","44,160\n",0.880\n,"60,665,551\n",N\n,Y\n,Y\n,Advanced\n
5,Japan\n,"1,522,400\n","5,079,916\n","5,450,654\n","40,255\n","43,193\n",0.909\n,"127,061,000\n",N\n,Y\n,Y\n,Advanced\n
6,United Kingdom\n,"1,189,400\n","2,830,764\n","3,254,845\n","42,378\n","48,727\n",0.922\n,"64,511,000\n",Y\n,Y\n,Y\n,Advanced\n
7,United States\n,"3,944,000\n","21,433,225\n","21,433,225\n","65,253\n","65,253\n",0.924\n,"318,523,000\n",Y\n,Y\n,Y\n,Advanced\n
8,European Union (2014)\n,"4,485,000\n","18,527,116\n","18,640,411\n","36,645\n","36,869\n",0.899\n,"505,570,700\n",N/A\n,Y\n,N/A\n,Emerging and Developing / Advanced[76]\n


In [40]:
countries = pd.read_csv('assets/countries_of_the_world.csv') #https://www.kaggle.com/fernandol/countries-of-the-world 1970-2017
countries.columns = countries.columns.str.lower()


In [41]:
countries[countries['country'].str.contains(' and |&')]

,country,region,population,area (sq. mi.),pop. density (per sq. mi.),coastline (coast/area ratio),net migration,infant mortality (per 1000 births),gdp ($ per capita),literacy (%),phones (per 1000),arable (%),crops (%),other (%),climate,birthrate,deathrate,agriculture,industry,service
7,Antigua & Barbuda,LATIN AMER. & CARIB,69108,443,"156,0","34,54","-6,15","19,46",11000.0,"89,0","549,9","18,18","4,55","77,27",2,"16,93","5,37","0,038","0,22","0,743"
25,Bosnia & Herzegovina,EASTERN EUROPE,4498976,51129,"88,0","0,04","0,31","21,05",6100.0,NaN,"215,4","13,6","2,96","83,44",4,"8,77","8,27","0,142","0,308","0,55"
172,Saint Kitts & Nevis,LATIN AMER. & CARIB,39129,261,"149,9","51,72","-7,11","14,49",8800.0,"97,0","638,9","19,44","2,78","77,78",2,"18,02","8,33","0,035","0,258","0,707"
174,St Pierre & Miquelon,NORTHERN AMERICA,7026,242,"29,0","49,59","-4,86","7,54",6900.0,"99,0","683,2","13,04",0,"86,96",NaN,"13,52","6,83",NaN,NaN,NaN
175,Saint Vincent and the Grenadines,LATIN AMER. & CARIB,117848,389,"303,0","21,59","-7,64","14,78",2900.0,"96,0","190,9","17,95","17,95","64,1",2,"16,18","5,98","0,1","0,26","0,64"
178,Sao Tome & Principe,SUB-SAHARAN AFRICA,193413,1001,"193,2","20,88","-2,72","43,11",1200.0,"79,3","36,2","6,25","48,96","44,79",2,"40,25","6,47","0,167","0,148","0,684"
204,Trinidad & Tobago,LATIN AMER. & CARIB,1065842,5128,"207,9","7,06","-10,83","24,31",9500.0,"98,6","303,5","14,62","9,16","76,22",2,"12,9","10,57","0,007","0,57","0,423"
208,Turks & Caicos Is,LATIN AMER. & CARIB,21152,430,"49,2","90,47","11,68","15,67",9600.0,"98,0","269,5","2,33",0,"97,67",2,"21,84","4,21",NaN,NaN,NaN
221,Wallis and Futuna,OCEANIA,16025,274,"58,5","47,08",NaN,NaN,3700.0,"50,0","118,6",5,25,70,2,NaN,NaN,NaN,NaN,NaN


In [42]:
countries

,country,region,population,area (sq. mi.),pop. density (per sq. mi.),coastline (coast/area ratio),net migration,infant mortality (per 1000 births),gdp ($ per capita),literacy (%),phones (per 1000),arable (%),crops (%),other (%),climate,birthrate,deathrate,agriculture,industry,service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN
5,Angola,SUB-SAHARAN AFRICA,12127071,1246700,"9,7","0,13",0,"191,19",1900.0,"42,0","7,8","2,41","0,24","97,35",NaN,"45,11","24,2","0,096","0,658","0,246"
6,Anguilla,LATIN AMER. & CARIB,13477,102,"132,1","59,80","10,76","21,03",8600.0,"95,0","460,0",0,0,100,2,"14,17","5,34","0,04","0,18","0,78"
7,Antigua & Barbuda,LATIN AMER. & CARIB,69108,443,"156,0","34,54","-6,15","19,46",11000.0,"89,0","549,9","18,18","4,55","77,27",2,"16,93","5,37","0,038","0,22","0,743"
8,Argentina,LATIN AMER. & CARIB,39921833,2766890,"14,4","0,18","0,61","15,18",11200.0,"97,1","220,4","12,31","0,48","87,21",3,"16,73","7,55","0,095","0,358","0,547"
9,Armenia,C.W. OF IND. STATES,2976372,29800,"99,9","0,00","-6,47","23,28",3500.0,"98,6","195,7","17,55","2,3","80,15",4,"12,07","8,23","0,239","0,343","0,418"


In [43]:
coordinates.to_json(r'data.json', orient="records")

In [44]:
len(coordinates)

245